<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/YOLOv5_Pytorch_modificaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 將Colab與google_drive綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 當前設備顯卡參數列表明細:

!nvidia-smi

Fri Jul 23 03:51:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# 從 Github 下載 YOLOv5 的資料庫

!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

Cloning into 'yolov5'...
remote: Enumerating objects: 8338, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 8338 (delta 24), reused 23 (delta 13), pack-reused 8288
Receiving objects: 100% (8338/8338), 9.24 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (5764/5764), done.
/content/yolov5
HEAD is now at 886f1c0 DDP after autoanchor reorder (#2421)


In [4]:
# 安裝必要的開發套件與環境
!pip install -qr requirements.txt
import torch

# 顯示圖片的套件
from IPython.display import Image, clear_output

# 下載模型和資料集的套件
from utils.google_utils import gdrive_download

# 顯示出當前訓練的配備
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 636 kB 13.8 MB/s 
Setup complete. Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16160MB, multi_processor_count=80)


In [5]:
# 下載資料集,並於 google_drive 取得壓縮檔的 id
# id 取得方法,到 google_drive 指定壓縮檔,滑鼠右鍵選擇取得連結,更改權限由限制 -> 知道連結的使用者,複製連結在筆記本上
# https://drive.google.com/file/d/這裡就是壓縮檔的id/view?usp=sharing

%cd /content
!gdown --id '15hePnoagt4UJ8bR6XSoKl7rX5OMygub0' -O dataset.zip

/content
Downloading...
From: https://drive.google.com/uc?id=15hePnoagt4UJ8bR6XSoKl7rX5OMygub0
To: /content/dataset.zip
274MB [00:04, 67.3MB/s]


In [6]:
# 解壓縮,注意壓縮檔名稱需與代碼一致
# 解壓縮完,於右邊選擇檔案,看有無已解壓縮的資料夾

!unzip -q dataset.zip 

In [7]:
# 確定類別檔案位置和類別數

%cd /content
%cat data.yaml

/content
train: ../train/images
val: ../valid/images

nc: 33
names: ['bacon', 'bagel', 'baguette', 'baozi', 'burger', 'cereal', 'chips', 'cong_you_bing', 'corn_soup', 'croissant', 'danbing', 'drinks', 'fantuan', 'fried_baozi', 'fried_egg', 'guotie', 'ham', 'hash_brown', 'hotdog', 'pancake', 'radish_cake', 'salad', 'sandwich', 'shaobing', 'sliced_bread', 'steamed_bread', 'tea_egg', 'teppanyaki_noodle', 'triangle_rice_ball', 'waffle', 'water_rice', 'xiaolongbao', 'you_tiao']

In [8]:
# 根據 data.yaml 來定義訓練的類別數
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [9]:
# 從 models 資料夾下抓取 YOLOv5 的架構
# 請注意因 YOLOv5 有4個版本,分別是 yolov5s, yolov5m, yolov5l, yolov5x, 請自行修改所需要的架構
# 並打印出 YOLOv5 的架構

%cat /content/yolov5/models/yolov5s.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C

In [10]:
# 導入 IPython 套件將方便寫入代碼
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [11]:
# YOLOv5 的架構,並可從下方代碼中改寫自己所想要的架構或參數,後寫入 yolov5.yaml
# 請注意因 YOLOv5 有4個版本,分別是 yolov5s, yolov5m, yolov5l, yolov5x, 請自行修改所需要的架構
# 若不需要修改,請保持官方設定

%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # 類別數
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# 以自己所需求的 epochs 數量去訓練 YOLOv5, 可自行修改所需要的 epochs數量,這裡是 1000 epochs
# 參數說明: img(輸入的圖片尺寸), batch( batch 大小), epochs( epochs 數量), data(yaml 的路徑),cfg (模型的架構),
#           weights(權重,可修改成自己已訓練的權重路徑), name(結果的名稱), nosave(只存最後的 checkpoint), cache(內存與圖像高速緩存)

%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 500 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
github: ⚠️ WARNING: code is out of date by 375 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.9.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='../data.yaml', device='', entity=None, epochs=500, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5s_results', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5s_results', single_cls=False, sync_bn=False, total_batch_size=16, weights='', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv5 logging with 'pip install wandb' (recommended)
Start Tensorboard with "tensorb

In [ ]:
# 使用 tensorboard 可視化套件
# 讀取 runs 資料夾下的結果

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# 導入繪圖與顯示套件
from utils.plots import plot_results

# 圖片路徑(如果不對記得修改), 繪圖參數
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)

In [ ]:
# 顯示正確 Test 照片和其標註的 label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results2/test_batch2_labels.jpg', width=900)

In [ ]:
# 顯示 augment 後 test 照片和其標註label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
# 已訓練完的權重會存在 runs 資料夾中

%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
# 進行 inference, 存放test資料夾下的照片
# 使用訓練時最好的權重, best.pt, (需要注意路徑是否正確)

%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ../test/images

In [ ]:
#顯示出 inference 後圖片, (需要注意路徑是否正確)

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#下載權重檔案到 google_drive 中

from google.colab import files
files.download('./runs/train/yolov5s_results/weights/best.pt')